In [1]:
import sys
sys.path.append("../../src/models")

In [2]:
import numpy as np
from actions import city_restrictions, costs

In [3]:
from simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population
from simulate_pandemic import main

In [4]:
from numpy.random import default_rng
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [5]:
from joblib import Parallel, delayed

In [6]:
def make_df(week, sim, action, data):
    df = pd.DataFrame(pd.Series(data[:, 1]).value_counts()).T
    df['simulation'] = sim
    df['week'] = week
    df['action'] = action
    df = df.rename(columns = {
                              -1 : 'removed',
                               0 : 'susceptible',
                               1 : 'exposed',
                               2 : 'infected',
                               3 : 'hospitalized'
                              }
                  )
    
    return df

In [27]:
def simulate_and_save(policy_number, 
                      policy,
                      n_sims,
                      step_size=7,
                      folder='generated_sims/'):
    dfs = []
    for sim in range(1, n_sims+1):
        
        sim_name = f'{policy_number}_{sim}'
                
        data, pop_matrix = main(gpickle_path=gpickle_path,
                                p_r=p_r,
                                policy=policy,
                                disable_tqdm=True,
                                days=step_size*len(policy),
                                step_size=step_size)
        weeks = (step_size/7) * len(policy)
        weekly_data = [data[i*7] for i in range(int(weeks))]
        df = pd.concat([make_df(week, sim_name, action, data)
                             for week, (data, action)
                             in enumerate(zip(weekly_data, policy))])
        #df.to_parquet(folder + f'{sim_name}.parquet')
        dfs += [df]
    return dfs

In [28]:
rng = default_rng(None)

prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

gpickle_path = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [29]:
n_policies = 6
n_sims=1

In [34]:
costs_keys = list(costs.keys())
costs_values = np.array(list(costs.values()))
p = (1 - costs_values) / (1 - costs_values).sum()

policy = [rng.choice(costs_keys,
                     size=int(364/14),
                     replace=True, p=p) for i in range(n_policies)]
assert len(set([tuple(p) for p in policy])) == len(policy)

In [35]:
res = Parallel(n_jobs=-1)(delayed(simulate_and_save)(i, pol, n_sims) 
                              for i, pol in tqdm(enumerate(policy),
                                                 total=len(policy)))

100%|██████████| 6/6 [00:00<00:00, 5991.86it/s]


In [36]:
res[0][0]

,susceptible,exposed,simulation,week,action,infected,removed,hospitalized
0,55460,32,0_1,0,Unrestricted,NaN,NaN,NaN
0,55422,30,0_1,1,Social Distancing,31.0,9.0,NaN
0,55288,87,0_1,2,Hard Quarantine,71.0,44.0,2.0
0,55134,119,0_1,3,Social Distancing,127.0,104.0,8.0
0,54810,215,0_1,4,Lockdown,220.0,238.0,9.0
0,54533,209,0_1,5,Unrestricted,278.0,459.0,13.0
0,53735,514,0_1,6,Social Distancing,473.0,753.0,17.0
0,52660,726,0_1,7,Light Quarantine,805.0,1280.0,21.0
0,51645,682,0_1,8,Lockdown,1037.0,2086.0,42.0
0,50933,521,0_1,9,Social Distancing,914.0,3061.0,63.0
